# Multiple Linear Regression, Part 1: 

## Feature Scaling and Model Assumptions

AKA - Topic 19 is too big to fit into one study group! We'll do part 1 today, focused on how we can better interpret the results of a linear regression model that includes multiple features, as well as some things to check to make sure our models are reliable.

### First: Set Up

In [ ]:
# Basic imports

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 50)

import matplotlib.pyplot as plt
import seaborn as sns

Credit data from https://www.kaggle.com/avikpaul4u/credit-card-balance

Target: `Balance`

In [ ]:
# Data
df = pd.read_csv('data/Credit.csv', 
                 usecols=['Income', 'Limit', 'Rating', 'Age', 'Balance'])

In [ ]:
df.head()

In [ ]:
df.describe()

## Multiple Linear Regression

Same as simple linear regression, but with more inputs!

In [ ]:
# Let's start with statsmodels
import statsmodels.api as sm

In [ ]:
# Define our X and y

X = None
y = None

In [ ]:
# Create our model


In [ ]:
# Look at our results


#### Observation time!

How'd we do? What looks different from the simple linear regression output? What in the world can we do with those coefficients?

- 


## Standardization, AKA Feature Scaling and Centering

Scaling data is the process of **increasing or decreasing the magnitude according to a fixed ratio.** You change the size but not the shape of the data. Often, this involves divivding features by their standard deviation.

Centering also does not change the shape of the data, but instead simply **removes the mean value  of each feature** so that each is centered around zero instead of their original mean.

The idea is that you can standardize data to be comparable, so that a model can interpret each individual feature more consistently.

**NOTE:** Feature Scaling is **NOT** the same as normalization!

Documentation: https://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler

#### [Standard Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

The most common method of scaling is standardization.  In this method we center the data, then we divide by the standard devation to enforce that the standard deviation of the variable is one.

#### [MinMax Scalar](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html)

From the documentation:

> This estimator scales and translates each feature individually such that it is in the given range on the training set, e.g. between zero and one.

#### [Robust Scaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html)

From the documentation:

> Scale features using statistics that are robust to outliers.
>
> This Scaler removes the median and scales the data according to the quantile range (defaults to IQR: Interquartile Range). The IQR is the range between the 1st quartile (25th quantile) and the 3rd quartile (75th quantile).

Aka like a standard scaler, but uses median and IQR variance instead of mean and standard deviation.

### Visualize it!

In [ ]:
# Importing some options so we can check out the differences between them
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

In [ ]:
# Instantiating our different scalers
stdscaler = StandardScaler()
minmaxscaler = MinMaxScaler()
robscaler = RobustScaler()

# Creating scaled versions of one column
X_scaled_std = stdscaler.fit_transform(X['Age'].values.reshape(-1, 1))
X_scaled_mm = minmaxscaler.fit_transform(X['Age'].values.reshape(-1, 1))
X_scaled_rob = robscaler.fit_transform(X['Age'].values.reshape(-1, 1))
# why fit_transform? We'll discuss in a second

# defining a dictionary of these things to better visualize
scalers = {'Original': X['Age'].values, 
           'Standard Scaler': X_scaled_std, 
           'Min Max Scaler': X_scaled_mm,
           'Robust Scaler': X_scaled_rob}

In [ ]:
# visualize it!
for title, data in scalers.items():
    plt.hist(data, bins=20)
    plt.title(f"{title}")
    plt.show()

Are you thinking to yourself: "Hey - these all look the same."

**EXACTLY**

The point is that you change the scale on the X axis WITHOUT changing the shape of the data!

### Why do we need to use feature scaling?

- In order to compare the magnitude of coefficients thus increasing the interpretability of coefficients
- Handling disparities in units
- Some models use euclidean distance in their computations
- Some models require features to be on equivalent scales
- In the machine learning space, it helps improve the performance of the model and reducing the values/models from varying widely
- Some algorithms are sensitive to the scale of the data

### Let's Try It

In [ ]:
# nstantiate a scaler and scale our data


In [ ]:
# Can transform that from an array to a df to explore


In [ ]:
# Now model on the scaled data


In [ ]:
# Look at our results


### Evaluate:

What changed?

- 


### Now that we're ready to look at our inputs - how do we interpret these coefficients, or those p-values in the summary?

Discuss:

- 


See anything you wouldn't expect? Could be because we haven't yet made our models reliable - aren't adhering to the assumptions of linear regression!

## The Assumptions of Linear Regression

Linear regression models have some underlying assumptions, mostly captured by the following points:

1. The true relationship is linear
2. No multicollinearity between independent variables
3. Errors are normally distributed with mean 0
4. Errors are homoskedastic (aka they have constant variance)
5. Errors are not correlated (no trends in error terms)


![the office "you're making some very dangerous assumptions" gif from gfycat](https://thumbs.gfycat.com/DarkParallelArizonaalligatorlizard-size_restricted.gif)

## Checking Each Assumption

### Linearity

Why do we assume linearity? Because, by modeling the relationship using _linear_ regression - if we don't think the relationship is linear, we probably should use a different model.

I'll note that linear regression can still handle curvature in the relationship using polynomial variables, interaction terms, etc (more on that in Topic 20) - but this assumption captures the idea that linear parameters (aka coefficients) can capture the relationship between X and y.

This assumption can be checked by using scatterplots - plotting the dependent variable against every independent variable.

In [ ]:
X_cols

In [ ]:
for x in X_cols:
    plt.scatter(df[x], df['Balance'])
    plt.title(f'Plot of Balance against {x}')
    plt.xlabel(x)
    plt.ylabel('Balance')
    plt.show()
    
# also plot sales against itself
plt.scatter(df.index, df['Balance'])
plt.hlines(df['Balance'].mean(), 0, df.index.max(), color='r')
plt.xlabel('Index Value')
plt.ylabel('Balance')
plt.title('Variance of Balance')
plt.show()

A quicker solution is to use Seaborn's `pairplot`.  

This lets us check for linearity and multicollinearity (the next assumption we'll check) at the same time.

In [ ]:
sns.pairplot(df)
plt.show()

#### Calculate Pearson's R Value

Pearson's R represents a correlation coefficient. 

In [ ]:
# check correlations just against sales
df.corr().Balance.sort_values(ascending=False)

### Multicollinearity

AKA when my X variables aren't actually independent - so that a model has trouble determining which change in what X variable is actually influencing `y`.

#### Directly Explore Correlations

In [ ]:
# check all correlations using the same Pearson's correlation coefficient
df.corr()

In [ ]:
# can also visualize it
sns.heatmap(df.corr(), annot=True)
plt.show()

#### Calculate the Variance Inflation Factor (VIF)

> "Variance inflation factor (VIF) is a measure of the amount of multicollinearity in a set of multiple regression variables. Mathematically, the VIF for a regression model variable is **equal to the ratio of the overall model variance to the variance of a model that includes only that single independent variable**. This ratio is calculated for each independent variable. A high VIF indicates that the associated independent variable is highly collinear with the other variables in the model."

-- Source: https://www.investopedia.com/terms/v/variance-inflation-factor.asp

In other words - how well does one of these X variables predict the others?

reference: https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

# defining a dataframe with just our X variables
X = df[X_cols]

# defining an empty dataframe to capture the VIF scores
vif = pd.DataFrame()

# For each column,run a variance_inflaction_factor against all other columns to get a VIF Factor score
vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]

# label the scores with their related columns
vif["features"] = df[X_cols].columns

vif

## Checking Residuals

By checking the residuals, aka the error between our actual y values and what we predicted, we can check that:

- Errors are normally distributed with mean 0
- Errors are homoskedastic (aka they have constant variance)
- Errors are not correlated (no trends in error terms)

In a nutshell:

<img src="images/error-dist.jpeg" width="550">  


#### Typical Residuals vs. Predictions plots:

- **The ideal scenario**

    - Random scatter
    - Scattered around 0
    - No identifiable trend
    
    <img src="images/normal-resid.png" width="550">  
    
- **Non-linear relationship**

    - Clear non-linear scatter, but
    - Identifiable trend
    - **Fix:** Introduce polynomial terms
    - **Fix:** Variable transformation
    
    <img src="images/polynomial-resid.png" width="550">

- **Autocorrelation**

    - Identifiable trend, or
    - Consecutively positive/negative residuals
    - **Fix:** Consider sequential analysis methods (which we'll discuss in phase 4)
    
    <img src="images/autocorrelation.png" width="550">

- **Heteroskedasticity**

    - The spread of residuals is different at different levels of the fitted values
    - **Fix:** Variable transformation (log)  
    
    <img src="images/heteroskedasticity.png" width="550">
    
The above plots were created using `seaborn.residplot` 

### Residual Plots

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()

lr.fit(X, y)

preds = lr.predict(X)

In [ ]:
from sklearn.metrics import r2_score

r2_score(y, preds)

### Normality

There are several ways to test for normality.

In [ ]:
residuals = y - preds

In [ ]:
plt.hist(residuals)
plt.show()

In [ ]:
# QQ plots are generally great tools for checking for normality.
import statsmodels.api as sm

fig = sm.qqplot(residuals, line = 'r')

If you're just doing/exploring simple linear regression:

In [ ]:
# note that these residplots only work for single variables

for x in X_cols:
    sns.residplot(x, "Balance", data=df)
    plt.show()

In [ ]:
# for our full model
plt.scatter(preds, residuals)
plt.axhline(y=0, color = 'red', label = '0')
plt.xlabel('predictions')
plt.ylabel('residuals')
plt.show()

## Other Potential Problems

- Outliers

    <img src='images/outliers.png' width=450>

- High Leverage Points 

    <img src='images/leverage.png' width=450>

## So Let's Fix It!

What problems have we identified?

- 


How can we potentially fix them?

- 


### This Is Part 1....

In Part 2, we'll cover how to validate our models, aka how to make sure they generalize well to unseen data. We'll also talk about the importance of _training_ our models only on the data that we want it to see!

We'll table how to transform categorical variables to be able to be used in models, and cover that with Polynomial Regression in our Topic 20 study group!

### Additional Resources

- [Excellent statistical writeup about how to interpret Linear Regression coefficients, and their p-values](https://statisticsbyjim.com/regression/interpret-coefficients-p-values-regression/)

- [Detecting Multicollinearity with VIF](https://www.geeksforgeeks.org/detecting-multicollinearity-with-vif-python/)

- [Penn State Stats on Influential Points (outliers, high leverage points)](https://online.stat.psu.edu/stat462/node/87/) - this resource also allows easy access to the rest of their material on regression

- [Statsmodels' Documentation: Check the influence of outliers](https://www.statsmodels.org/devel/generated/statsmodels.stats.outliers_influence.OLSInfluence.html)

- [Long blog post on regression diagnostics with implementation in python](http://songhuiming.github.io/pages/2016/12/31/linear-regression-in-python-chapter-2/)

- [Statistics by Jim: Linear Regression Assumptions](https://statisticsbyjim.com/regression/ols-linear-regression-assumptions/)